# Stock Prediction Using Random Forest Regression

Goal: Use yfinance API and Sklearn to predict stock prices

Libraries used: 
* Pandas: https://pandas.pydata.org/docs/
* Numpy: https://numpy.org/doc/
* Sklearn: https://scikit-learn.org/stable/index.html
* yfinance: https://pypi.org/project/yfinance/
* Matplotlib: https://matplotlib.org/
* Statistics: an original python package - https://docs.python.org/3/library/statistics.html

I am not a financial professional and this is just for educational purposes and should not be used for financial advice.

## Import libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import statistics

## Get Stock Data

In [2]:
# Define Function
def fetch_data(ticker, period='1y'):
    # Download data
    stock_data = yf.download(ticker, period=period)

    # Return df
    return stock_data

## Calculate RSI

RSI is a technical indicator that indicates buying trends and can define a stock as over or underbought. 

In [3]:
# Define RSI Function
def calculate_rsi(data, window=14):
    # Calculate difference between each row
    delta = data['Close'].diff()

    # Isolate where the stock gains and lost 
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    # Calculate the rolling average using a window of 14 which is standard for RSI
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()

    # Calculate RSI
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    # Return RSI
    return rsi

## Calculate Bollinger Bands

In [4]:
# Calculate Bollinger Bands
def calculate_bollinger_bands(data, window=20):
    # Calculate rolling average and standard deviation
    sma = data['Close'].rolling(window=window).mean()
    std = data['Close'].rolling(window=window).std()

    # Calculate bolling bands 
    bollinger_upper = sma + (std * 2)
    bollinger_lower = sma - (std * 2)

    # Return values
    return bollinger_upper, bollinger_lower

## Calculate SMA

In [5]:
# Define SMA function
def calculate_sma(data, window):
    # Calculate SMA
    sma = data['Close'].rolling(window=window).mean()

    # Return SMA
    return sma

## Calculate EMA

In [6]:
# Define EMA function
def calculate_ema(data):
    span = 14 
    # Perform exponential weighted functions
    ema = data['Close'].ewm(span=span, adjust=False).mean()

    # Return value
    return ema

## Add features

In [7]:
# Functions for first group of features
def add_indicators1(data):
    # Call RSI
    data['RSI'] = calculate_rsi(data)
    # Call EMA
    data['ema'] = calculate_ema(data)
    # Call SMA with window = 20
    data['SMA_20'] = calculate_sma(data, 20)
    # Call SMA with window = 50
    data['SMA_50'] = calculate_sma(data, 50)
    # Call Bolinger Bands Function
    data['Bollinger_Upper'], data['Bollinger_Lower'] = calculate_bollinger_bands(data)
    # Gather Volume data
    data['Volume'] = data['Volume']
    data = data.dropna()
    return data

## Predict Stock Prices and Errors

In [8]:
# Define function
def return_accuracy_measures(data, features):
    # Aggregate features
    X = data[features]
    y = data['Close'].shift(-1).dropna()
    X = X[:-1]

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # Evaluation
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    # Return errors
    return mse, mae, r2

In [9]:
# Define function to get the accuracy as well as stock data all in one
def get_accuracy(ticker, features):
    data = fetch_data(ticker)
    data = add_indicators1(data)
    mse, mae, r2 = return_accuracy_measures(data, features)
    return mse, mae, r2

## Test the model

In [10]:
# Select S&P 500 as test ticker
ticker = '^SPX'

# First Group of Features
features1 = ['RSI', 'SMA_20', 'SMA_50', 'Bollinger_Upper', 'Bollinger_Lower', 'Volume']
mse, mae, r2 = get_accuracy(ticker, features1)

# Model Metrics
print(f"MSE: {mse}, MAE: {mae}, R^2: {r2}")

[*********************100%%**********************]  1 of 1 completed


MSE: 1880.7994288153352, MAE: 34.08260206269057, R^2: 0.986498715593811


## Applying the model to a diverse range of stocks

In [11]:
# Define arrays for errors
mse_array = []
mae_array = []
r2_array = []

I selected these 22 stocks because they provide a diverse group of stocks from the S&P 500, allowing the model to prove its accuracy across industries. 

In [12]:
# Create a diverse array of tickers
# there are 2 from each of the 11 sectors of the S&P 500
tickers_array = ['LLY', 'UNH', 'MSFT', 'AAPL', 'V', 'JPM', 'NKE', 'BKNG', 'VZ', 'DIS', 'GE', 'CAT', 'PG', 'WMT', 'XOM', 'VLO', 'NEE', 'SRE', 'PLD', 'AMT', 'LIN', 'FCX']
len(tickers_array)

22

In [13]:
# Iterate through the tickers in the array and get the accuracy of the model across them
for ticker in tickers_array:
    mse_ticker, mae_ticker, r2_ticker = get_accuracy(ticker, features1)
    mse_array.append(mse_ticker)
    mae_array.append(mae_ticker)
    r2_array.append(r2_ticker)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [14]:
# Calculate the mean error from the arrays 
mse_mean1 = statistics.fmean(mse_array)
mae_mean1 = statistics.fmean(mae_array)
r2_mean1 = statistics.fmean(r2_array)

In [15]:
# Print the results from the first model
results1 = 'Means for MSE: ', mse_mean1, ', MAE: ', mae_mean1, ', R^2: ', r2_mean1, '. This is with features: RSI, SMA_20, SMA_50, Bollinger_Upper, Bollinger_Lower, Volume.'
print(results1)

('Means for MSE: ', 205.26539453335053, ', MAE: ', 4.778322167830034, ', R^2: ', 0.943526662663957, '. This is with features: RSI, SMA_20, SMA_50, Bollinger_Upper, Bollinger_Lower, Volume.')


## Second Model

In [16]:
# Add ATR to the model
features2 = []
features2 = features1
features2.append('ATR')

In [17]:
# Define function to calculate ATR
def calculate_atr(data):
    # Calculate True Range
    data['High-Low'] = data['High'] - data['Low']
    data['High-PrevClose'] = (data['High'] - data['Close'].shift(1)).abs()
    data['Low-PrevClose'] = (data['Low'] - data['Close'].shift(1)).abs()
    data['TR'] = data[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)

    # Calculate ATR by averging the highest mean from each day in the 14 day period
    atr_period = 14
    ATR = data['TR'].rolling(window=atr_period).mean()
    return ATR

In [18]:
# Add ATR to the second group of fetches of data
def add_indicators2(data):
    data['RSI'] = calculate_rsi(data)
    data['ema'] = calculate_ema(data)
    data['SMA_20'] = calculate_sma(data, 20)
    data['SMA_50'] = calculate_sma(data, 50)
    data['ATR'] = calculate_atr(data)
    data['Bollinger_Upper'], data['Bollinger_Lower'] = calculate_bollinger_bands(data)
    data['Volume'] = data['Volume']
    data = data.dropna()
    return data

In [19]:
# Define the function
def test(ticker):
    data = fetch_data(ticker)
    data = add_indicators2(data)

    # Check that ATR was added successfully
    print(data.head())

In [20]:
test('^SPX')

[*********************100%%**********************]  1 of 1 completed

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2023-10-04  4233.830078  4268.500000  4220.479980  4263.750000  4263.750000   
2023-10-05  4259.310059  4267.129883  4225.910156  4258.189941  4258.189941   
2023-10-06  4234.790039  4324.100098  4219.549805  4308.500000  4308.500000   
2023-10-09  4289.020020  4341.729980  4283.790039  4335.660156  4335.660156   
2023-10-10  4339.750000  4385.459961  4339.640137  4358.240234  4358.240234   

                Volume        RSI          ema       SMA_20       SMA_50  \
Date                                                                       
2023-10-04  3777600000  20.141090  4322.673658  4374.270508  4435.161797   
2023-10-05  3581470000  22.934167  4314.075829  4364.622998  4428.990596   
2023-10-06  3902030000  31.962850  4313.332385  4357.173486  4424.412393   
2023-10-09  3174630000  37.096388  4316.309422  4349.583496  4419.

## Get the second accuracy 

In [21]:
def get_accuracy2(ticker, features):
    data = fetch_data(ticker)
    data = add_indicators2(data)
    mse2, mae2, r22 = return_accuracy_measures(data, features)
    return mse2, mae2, r22

In [22]:
mse_array2 = []
mae_array2 = []
r2_array2 = []

In [23]:
for ticker in tickers_array:
    mse_ticker2, mae_ticker2, r2_ticker2 = get_accuracy2(ticker, features2)
    mse_array2.append(mse_ticker2)
    mae_array2.append(mae_ticker2)
    r2_array2.append(r2_ticker2)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [24]:
mse_mean2 = statistics.fmean(mse_array2)
mae_mean2 = statistics.fmean(mae_array2)
r2_mean2 = statistics.fmean(r2_array2)

In [25]:
results2 = 'Means for MSE: ', mse_mean2, ', MAE: ', mae_mean2, ', R^2: ', r2_mean2, '. This is with features: RSI, SMA_20, SMA_50, Bollinger_Upper, Bollinger_Lower, Volume, ATR.'
print(results2)

('Means for MSE: ', 227.62813201608333, ', MAE: ', 4.920219143552422, ', R^2: ', 0.9437758026260433, '. This is with features: RSI, SMA_20, SMA_50, Bollinger_Upper, Bollinger_Lower, Volume, ATR.')
